import pandas as pd

In [1]:
import pandas as pd

In [2]:
asty = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Asty Hotel.csv', error_bad_lines=False);
hypnos = pd.read_csv(r'C:\Users\Dell\Desktop\Hotel\Hypnos by Bed n Mix.csv', error_bad_lines=False);

tripadv_review = pd.concat([asty, hypnos])
tripadv_review

,r_points,r_text,place
0,50,Like having a family in Nicosia. We loved stay...,Asty Hotel
1,50,"OK, it’s a good value 2 star hotel, it is mode...",Asty Hotel
2,50,"Highly recommended hotel, I booked this hotel ...",Asty Hotel
3,50,Large room. Very clean. Friendly staff. Good b...,Asty Hotel
4,40,The hotel has a homey feel and is located in a...,Asty Hotel
...,...,...,...
28,50,"Everything i can think about was perfect, from...",Hypnos by Bed n Mix
29,50,Everything was perfect. I will definitely reco...,Hypnos by Bed n Mix
30,50,I had no idea that Nicosia does not have many ...,Hypnos by Bed n Mix
31,50,This new place to stay is perfect from all aps...,Hypnos by Bed n Mix


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [6]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0     [like, having, family, nicosia, loved, staying...
1     [good, value, star, hotel, modest, little, fas...
2     [highly, recommended, hotel, booked, hotel, bi...
3     [large, room, clean, friendly, staff, good, br...
4     [hotel, homey, feel, located, quiet, neighborh...
                            ...                        
28    [think, perfect, cleaness, having, mobile, pho...
29    [perfect, definitely, recommend, number, accom...
30    [idea, nicosia, great, places, stay, lucky, ho...
31    [place, stay, perfect, apsects, nice, comforta...
32    [travel, cyprus, approx, months, normally, sta...
Name: r_text, Length: 442, dtype: object

In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]]


In [16]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=500,
                                           passes=5,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 25,
                                           eval_every = None)

In [17]:
from pprint import pprint

In [18]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.014*"room" + 0.011*"hotel" + 0.009*"good" + 0.007*"breakfast" + '
  '0.006*"clean" + 0.006*"area" + 0.006*"service" + 0.006*"night" + '
  '0.006*"rooms" + 0.005*"staff"'),
 (1,
  '0.044*"hotel" + 0.020*"room" + 0.014*"free" + 0.014*"stay" + '
  '0.014*"breakfast" + 0.013*"staff" + 0.012*"nice" + 0.012*"nicosia" + '
  '0.011*"asty" + 0.011*"great"'),
 (2,
  '0.043*"hotel" + 0.017*"city" + 0.017*"staff" + 0.013*"stay" + 0.011*"free" '
  '+ 0.011*"asty" + 0.011*"good" + 0.010*"nicosia" + 0.009*"friendly" + '
  '0.009*"room"'),
 (3,
  '0.029*"hotel" + 0.018*"staff" + 0.016*"room" + 0.014*"free" + '
  '0.013*"breakfast" + 0.011*"clean" + 0.009*"friendly" + 0.009*"stay" + '
  '0.009*"nicosia" + 0.008*"good"'),
 (4,
  '0.037*"hotel" + 0.037*"free" + 0.019*"room" + 0.016*"good" + '
  '0.014*"breakfast" + 0.012*"clean" + 0.010*"asty" + 0.010*"nicosia" + '
  '0.010*"shuttle" + 0.009*"rooms"')]


In [19]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.029935  0.019743       1        1  32.883499
1     -0.040160  0.010969       2        1  31.923252
3     -0.021454  0.031474       3        1  20.594282
2     -0.007953 -0.073467       4        1  10.898923
0      0.099502  0.011281       5        1   3.700038, topic_info=          Term        Freq       Total Category  logprob  loglift
5        hotel  780.000000  780.000000  Default  30.0000  30.0000
234       city  169.000000  169.000000  Default  29.0000  29.0000
80        room  363.000000  363.000000  Default  28.0000  28.0000
31       staff  260.000000  260.000000  Default  27.0000  27.0000
23        good  246.000000  246.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
38       value    2.290305   66.068962   Topic5  -5.8148  -0.0652
72        nice    2.417323  153.983444   Topic5  -5.7609  -0.8574
12        stay    2.438807  220.568298   Topic5  -5.7520  -1.2079
195      great    2.292027  174.058594   Topic5  -5.8141  -1.0331
104  available    2.045125   95.502663   Topic5  -5.9281  -0.5469

[417 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1663      4  0.652787      academy
1678      2  0.355072   accessible
1678      3  0.355072   accessible
1678      5  0.355072   accessible
1043      3  0.896628  accommodate
...     ...       ...          ...
1677      1  0.259500     wireless
1677      4  0.778499     wireless
761       2  0.311549         wise
761       4  0.623098         wise
1020      3  0.715858        worry

[724 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 4, 3, 1])